###### Content under Creative Commons Attribution license CC-BY 4.0, code under BSD 3-Clause License © 2017 L.A. Barba, N.C. Clementi

# Catch things in motion

This module of the _Engineering Computations_ course is our launching pad to investigate _change_, _motion_, _dynamics_, using computational thinking, Python, and Jupyter.

The foundation of physics and engineering is the subject of **mechanics**: how things move around, when pushed around. Or pulled… in the beginning of the history of mechanics, Galileo and Newton seeked to understand how and why objects fall under gravity.

This first lesson will explore motion by analyzing images and video, to learn about velocity and acceleration.

## Acceleration of a fallling ball

Let's start at the beginning. Suppose you want to use video capture of a falling ball to _compute_ the acceleration of gravity. Could you do it? With Python, of course you can!

Here is a neat video we found online, produced over at MIT several years ago. It shows a ball being dropped in front of a metered panel, while lit by a stroboscopic light. Watch the video!

In [ ]:
from IPython.display import YouTubeVideo
vid = YouTubeVideo("xQ4znShlK5A")
display(vid)

We learn on the video that the marks on the panel are every $0.25\rm{m}$, and on the [website](http://techtv.mit.edu/collections/physicsdemos/videos/831-strobe-of-a-falling-ball) they say that the strobe light flashes at about 15 Hz (that's 15 times per second). The final [image on Flickr](https://www.flickr.com/photos/physicsdemos/3174207211), however, notes that the strobe fired 16.8 times per second. So we have some uncertaintly already!

You can find several toolkits for handling images and video with Python; we'll start with a simple one called [`imageio`](https://imageio.github.io). Import this library like any other, and let's load `numpy` and `pyplot` while we're at it.


In [ ]:
import imageio
import numpy
from matplotlib import pyplot

With `get_reader()`, you can read a video from its source into a _Reader_ object. You don't need to worry too much about the technicalities here—we'll walk you through it all—but check the type, the length (for a video, that's number of frames, and notice you can get info, like the frames-per-second, using `get_meta_data()`.

In [ ]:
reader = imageio.get_reader('http://techtv.mit.edu/videos/831-strobe-of-a-falling-ball/download.mp4')

In [ ]:
type(reader)

In [ ]:
len(reader)

In [ ]:
fps = reader.get_meta_data()['fps']
print(fps)

##### Note:

You may get this error after calling `get_reader()`:

NeedDownloadError: Need ffmpeg exe. You can obtain it with either:
  - install using conda: `conda install ffmpeg -c conda-forge`
  - download by calling: `imageio.plugins.ffmpeg.download()`

If you do, follow the tips to install the needed `ffmpeg` tool.

In [ ]:
%matplotlib notebook

In [ ]:
image = reader.get_data(1100)
pyplot.imshow(image, interpolation='nearest');

In [ ]:
fig = pyplot.figure()

pyplot.imshow(image, interpolation='nearest')

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 

connectId = fig.canvas.mpl_connect('button_press_event', onclick)


In [ ]:
coords

In [ ]:
y = numpy.array(coords)[:,1]*0.25/gap_lines
y

In [ ]:
dt = 1 / fps

In [ ]:
gap_lines = y[1] - y[0]
gap_lines

\begin{equation}
\bar{v}_i = \frac{y_{i+1}-y_i}{\Delta t}
\end{equation}

In [ ]:
v = (y[1:] - y[:-1])  *15
v

In [ ]:
a = (v[1:] - v[:-1]) *15
a

In [ ]:
a[1:].mean()

In [ ]:
numpy.std(a[1:], ddof=1)

In [ ]:
7.58-2.83

In [ ]:
reader = imageio.get_reader('Sample.mp4')

In [ ]:
fps = reader.get_meta_data()['fps']
print(fps)

In [ ]:
image = reader.get_data(0)

In [ ]:
image.shape

In [ ]:
type(image)

In [ ]:
image1 = reader.get_data(10)
image2 = reader.get_data(20)
pyplot.imshow(image1, interpolation='nearest', alpha=0.9)
pyplot.imshow(image2, interpolation='nearest', alpha=0.7)


**Notes**

`iter_data()`
Iterate over all images in the series. (Note: you can also iterate over the reader object.)

Iterate over frames in a movie:

```Python
for i, im in enumerate(reader):```

### Capture clicks from movie frames

We'd like to capture the coordinates of mouse clicks on a sequence of images, so that we may have the positions of a moving ball caught on video. We know how to capture the coordinates of mouse clicks, so the challenge is to get consecutive frames of the video displayed for us, to click on the ball position each time. 

Widgets to the rescue! There are currently [10 different widget types](http://ipywidgets.readthedocs.io/en/stable/examples/Widget%20List.html) included in the `ipywidgets` library. The `BoundedIntText()` widget shows a text box with an integer value that can be stepped from a minimum to a maximum value by clicking up/down arrows. Stepping through frames with this widget, and clicking on the ball position each time, gets us what we want.

Digitizing the ball positions in this way is a bit tedious. But this could be a realistic scenario: you captured video of a moving object, and you need to get position data from the video frames. Unless you have some fancy motion-capture equipment, this will do the trick.

Let's load the Jupyter widgets:

In [ ]:
from ipywidgets import widgets

In [ ]:
selector = widgets.BoundedIntText(value=0, min=0, max=38, step=1,
    description='Frame:',
    disabled=False)

coords = []
def onclick(event):
    '''Capture the x,y coordinates of a mouse click on the image'''
    ix, iy = event.xdata, event.ydata
    coords.append([ix, iy]) 


def catchclick(frame):
    image = reader.get_data(frame)
    pyplot.imshow(image, interpolation='nearest');



fig = pyplot.figure()
#fig.add_subplot()

connectId = fig.canvas.mpl_connect('button_press_event', onclick)

widgets.interact(catchclick, frame=selector);

In [ ]:
coords

In [ ]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../../style/custom.css'
HTML(open(css_file, "r").read())